### Creating a virtual environment with every required libraries installed in it.
python -m venv .venv

#### Activating it
.venv/Scripts/Activate

# Cross-Country Analysis of Economic Growth Trends (1970–2024)

## Phase 1: Preprocessing & Feature Engineering

### Objectives:
- Clean GDP dataset
- Handle missing values
- Restructure dataset for time-series analysis
- Prepare features for growth rate computation

#### Before performing manipulations in Python, I deleted the first two rows from the dataset because they were not needed.

## Importing Libraries

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler 

## Loading the Dataset

In [23]:
# Load dataset
file_path = r"C:\Projects\DAP\gdp-growth-analysis\data\Download-GDPcurrent-USD-countries.xlsx"
df_raw = pd.read_excel(file_path)

# Create working copy
df = df_raw.copy()

df.head()

,CountryID,Country,IndicatorName,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,4,Afghanistan,Final consumption expenditure,1.691109e+09,1.826664e+09,1.544444e+09,1.637778e+09,1.984444e+09,2.173333e+09,2.248889e+09,...,1.905434e+10,1.841552e+10,1.859587e+10,2.082703e+10,2.138090e+10,2.286309e+10,1.663770e+10,1.693301e+10,2.009309e+10,2.708702e+10
1,4,Afghanistan,Household consumption expenditure (including N...,1.577102e+09,1.703519e+09,1.440325e+09,1.527366e+09,1.850662e+09,2.026817e+09,2.097279e+09,...,1.496906e+10,1.429584e+10,1.460787e+10,1.688326e+10,1.721976e+10,1.844500e+10,1.341413e+10,1.383642e+10,1.662669e+10,2.212882e+10
2,4,Afghanistan,General government final consumption expenditure,1.140071e+08,1.231456e+08,1.041196e+08,1.104117e+08,1.337824e+08,1.465165e+08,1.516101e+08,...,4.085273e+09,4.119686e+09,3.987993e+09,3.943771e+09,4.161138e+09,4.418088e+09,3.223572e+09,3.096592e+09,3.466403e+09,4.958199e+09
3,4,Afghanistan,Gross capital formation,9.555556e+07,9.999998e+07,1.044445e+08,1.266666e+08,1.888889e+08,2.311111e+08,3.488889e+08,...,2.711463e+09,2.331712e+09,2.640511e+09,2.689519e+09,2.434404e+09,2.289180e+09,1.968903e+09,2.362752e+09,2.490757e+09,2.041052e+09
4,4,Afghanistan,Gross fixed capital formation (including Acqui...,9.555556e+07,9.999998e+07,1.044445e+08,1.266666e+08,1.888889e+08,2.311111e+08,3.488889e+08,...,2.711463e+09,2.331712e+09,2.640511e+09,2.689519e+09,2.434404e+09,2.289180e+09,1.968903e+09,2.362752e+09,2.490757e+09,2.041052e+09


## Data Overview & Structure Inspection

In [24]:
print("Shape of dataset:", df.shape)
print("\nColumn Names:")
print(df.columns)

print("\nDataset Info:")
df.info()

print("\nMissing Values per Column:")
print(df.isna().sum())

Shape of dataset: (3715, 58)

Column Names:
Index([    'CountryID',       'Country', 'IndicatorName',            1970,
                  1971,            1972,            1973,            1974,
                  1975,            1976,            1977,            1978,
                  1979,            1980,            1981,            1982,
                  1983,            1984,            1985,            1986,
                  1987,            1988,            1989,            1990,
                  1991,            1992,            1993,            1994,
                  1995,            1996,            1997,            1998,
                  1999,            2000,            2001,            2002,
                  2003,            2004,            2005,            2006,
                  2007,            2008,            2009,            2010,
                  2011,            2012,            2013,            2014,
                  2015,            2016,            2017

In [25]:
print("Unique Indicators:")
print(df["IndicatorName"].unique())

Unique Indicators:
['Final consumption expenditure'
 'Household consumption expenditure (including Non-profit institutions serving households)'
 'General government final consumption expenditure'
 'Gross capital formation'
 'Gross fixed capital formation (including Acquisitions less disposals of valuables)'
 'Exports of goods and services' 'Imports of goods and services'
 'Gross Domestic Product (GDP)'
 'Agriculture, hunting, forestry, fishing (ISIC A-B)'
 'Mining, Manufacturing, Utilities (ISIC C-E)' 'Manufacturing (ISIC D)'
 'Construction (ISIC F)'
 'Wholesale, retail trade, restaurants and hotels (ISIC G-H)'
 'Transport, storage and communication (ISIC I)'
 'Other Activities (ISIC J-P)' 'Total Value Added'
 'Changes in inventories']


In [26]:
df_gdp = df[df["IndicatorName"].str.contains("GDP", case=False)]

df_gdp.head()

,CountryID,Country,IndicatorName,1970,1971,1972,1973,1974,1975,1976,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
7,4,Afghanistan,Gross Domestic Product (GDP),1.748887e+09,1.831109e+09,1.595555e+09,1.733333e+09,2.155556e+09,2.366667e+09,2.555556e+09,...,1.869957e+10,1.822435e+10,1.903430e+10,1.885632e+10,1.937236e+10,1.998301e+10,1.516092e+10,1.417499e+10,1.633976e+10,1.828316e+10
24,8,Albania,Gross Domestic Product (GDP),2.265577e+09,2.330585e+09,2.397526e+09,2.466543e+09,2.537016e+09,2.610110e+09,2.685746e+09,...,1.147017e+10,1.198867e+10,1.325824e+10,1.537951e+10,1.558511e+10,1.524146e+10,1.803199e+10,1.901725e+10,2.349124e+10,2.703747e+10
41,12,Algeria,Gross Domestic Product (GDP),5.155121e+09,5.363676e+09,7.176428e+09,9.229622e+09,1.325976e+10,1.555555e+10,1.775003e+10,...,1.874939e+11,1.807638e+11,1.898809e+11,1.945545e+11,1.934597e+11,1.648734e+11,1.862317e+11,2.256385e+11,2.476262e+11,2.669723e+11
58,20,Andorra,Gross Domestic Product (GDP),9.947679e+07,1.131285e+08,1.435060e+08,1.908638e+08,2.360555e+08,2.785143e+08,2.875884e+08,...,2.789880e+09,2.896613e+09,3.000160e+09,3.218419e+09,3.155152e+09,2.890998e+09,3.324642e+09,3.380613e+09,3.785063e+09,4.039838e+09
75,24,Angola,Gross Domestic Product (GDP),3.806982e+09,4.007445e+09,4.102155e+09,5.016276e+09,5.626961e+09,4.147433e+09,3.980907e+09,...,1.316610e+11,1.147637e+11,1.398346e+11,1.141892e+11,9.467123e+10,6.652065e+10,8.437511e+10,1.424029e+11,1.143358e+11,1.171594e+11


In [27]:
print("Shape after GDP filter:", df_gdp.shape)

Shape after GDP filter: (220, 58)


In [30]:
df_gdp.dtypes

CountryID          int64
Country           object
IndicatorName     object
1970             float64
1971             float64
1972             float64
1973             float64
1974             float64
1975             float64
1976             float64
1977             float64
1978             float64
1979             float64
1980             float64
1981             float64
1982             float64
1983             float64
1984             float64
1985             float64
1986             float64
1987             float64
1988             float64
1989             float64
1990             float64
1991             float64
1992             float64
1993             float64
1994             float64
1995             float64
1996             float64
1997             float64
1998             float64
1999             float64
2000             float64
2001             float64
2002             float64
2003             float64
2004             float64
2005             float64
2006             float64


In [31]:
df_gdp.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 220 entries, 7 to 3706
Data columns (total 58 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CountryID      220 non-null    int64  
 1   Country        220 non-null    object 
 2   IndicatorName  220 non-null    object 
 3   1970           187 non-null    float64
 4   1971           187 non-null    float64
 5   1972           187 non-null    float64
 6   1973           187 non-null    float64
 7   1974           187 non-null    float64
 8   1975           187 non-null    float64
 9   1976           187 non-null    float64
 10  1977           187 non-null    float64
 11  1978           187 non-null    float64
 12  1979           187 non-null    float64
 13  1980           187 non-null    float64
 14  1981           187 non-null    float64
 15  1982           187 non-null    float64
 16  1983           187 non-null    float64
 17  1984           187 non-null    float64
 18  1985          

In [32]:
year_columns = [col for col in df_gdp.columns if str(col).isdigit()]

df_gdp[year_columns].describe()

,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
count,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,1.870000e+02,...,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02,2.120000e+02
mean,1.829734e+10,2.013764e+10,2.317109e+10,2.819060e+10,3.193288e+10,3.571391e+10,3.858786e+10,4.344857e+10,5.147409e+10,5.923482e+10,...,3.562585e+11,3.621300e+11,3.853496e+11,4.092343e+11,4.151454e+11,4.044198e+11,4.620231e+11,4.807423e+11,5.021776e+11,5.224912e+11
std,8.866019e+10,9.628708e+10,1.075792e+11,1.239086e+11,1.336752e+11,1.470199e+11,1.606235e+11,1.796533e+11,2.098045e+11,2.344635e+11,...,1.555933e+12,1.597216e+12,1.685309e+12,1.806712e+12,1.868176e+12,1.868927e+12,2.132042e+12,2.254208e+12,2.356668e+12,2.460942e+12
min,2.585174e+06,2.747866e+06,3.028236e+06,3.780359e+06,4.013522e+06,3.847741e+06,3.797642e+06,3.546258e+06,3.853304e+06,3.998563e+06,...,3.619381e+07,4.130612e+07,4.528888e+07,4.816892e+07,5.378957e+07,5.246941e+07,6.145732e+07,5.418711e+07,5.054876e+07,5.643602e+07
25%,2.194021e+08,2.445365e+08,2.850735e+08,3.614351e+08,4.097892e+08,4.928473e+08,5.305898e+08,5.819454e+08,6.696585e+08,7.593423e+08,...,6.064012e+09,5.973133e+09,5.843874e+09,6.326665e+09,6.369348e+09,6.217613e+09,7.079508e+09,7.059586e+09,7.542800e+09,8.480262e+09
50%,1.067267e+09,1.213650e+09,1.300600e+09,1.605731e+09,2.127503e+09,2.398801e+09,2.680976e+09,2.791325e+09,3.098138e+09,3.697940e+09,...,2.337504e+10,2.390788e+10,2.546683e+10,2.667462e+10,2.759673e+10,2.541755e+10,3.093001e+10,3.179752e+10,3.486088e+10,3.690406e+10
75%,6.017784e+09,6.933917e+09,8.145223e+09,9.409526e+09,1.238624e+10,1.315209e+10,1.426080e+10,1.602731e+10,1.834957e+10,2.238019e+10,...,1.779699e+11,1.701428e+11,1.878835e+11,1.966976e+11,1.846153e+11,1.664257e+11,1.977757e+11,2.255319e+11,2.445303e+11,2.533489e+11
max,1.073303e+12,1.164850e+12,1.279110e+12,1.425376e+12,1.545243e+12,1.684904e+12,1.873412e+12,2.081826e+12,2.351599e+12,2.627333e+12,...,1.829502e+13,1.880491e+13,1.961210e+13,2.065652e+13,2.154000e+13,2.137530e+13,2.372560e+13,2.605460e+13,2.781150e+13,2.929800e+13


In [33]:
missing_summary = df_gdp[year_columns].isna().sum()
missing_summary.sort_values(ascending=False).head(10)

1970    33
1971    33
1972    33
1973    33
1974    33
1975    33
1976    33
1977    33
1978    33
1979    33
dtype: int64

In [34]:
print("Duplicate countries:", df_gdp["Country"].duplicated().sum())

Duplicate countries: 0
